# Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import cv2
from keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import random

## Image Directory

In [ ]:
directory= "/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset"

In [ ]:
label = []
path = []
for dir_name, _, filenames in os.walk(directory):
    for filename in filenames:
        if dir_name.split()[-1] != 'GT':
            if os.path.splitext(filename)[-1] == '.png':
                label.append(os.path.split(dir_name)[-1])
                path.append(os.path.join(dir_name, filename))

df = pd.DataFrame(columns=['path', 'label'])
df['path'] = path
df['label'] = label

## Dataset Samples

In [ ]:
i = 0
plt.figure(figsize=(15,12))
for unique_label in df['label'].unique():
    plt.subplot(3, 3, i+1)
    plt.imshow(plt.imread(df[df['label']==unique_label].iloc[0,0]))
    plt.title(unique_label)
    plt.axis('off')
    i+=1

## Train-Validation-Test Split

In [ ]:
train, test = train_test_split(df, test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.2, random_state=21)

In [ ]:
# Görüntü ve etiketleri yükleme ve preprocessing fonksiyonu
def process_path(file_path, label):
    # Görüntüyü yükleme
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [590, 445])
    img = img / 255.0  # Normalizasyon

    # Label one hot encoder
    label = tf.one_hot(label, depth=len(label_encoder.classes_))
    return img, label

## Label Encoding

In [ ]:
# label encoding
label_encoder = LabelEncoder()
train['label'] = label_encoder.fit_transform(train['label'])
val['label'] = label_encoder.transform(val['label'])
test['label'] = label_encoder.transform(test['label'])

## Tensorflow Dataset Oluşturma

Hızlı bir train süreci için verilerin tensorflow dataset formatına çevrilmesi önem arz etmektedir.

In [ ]:
# TensorFlow Dataset oluşturma
def dataframe_to_dataset(df):
    paths = df['path'].values
    labels = df['label'].values
    dataset = tf.data.Dataset.from_tensor_slices((paths, labels))
    return dataset.map(process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)

train_ds = dataframe_to_dataset(train)
val_ds = dataframe_to_dataset(val)
test_ds = dataframe_to_dataset(test)

# Veri setlerini konfigüre etme
train_ds = train_ds.shuffle(buffer_size=1000).batch(64).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
val_ds = val_ds.batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

## Model
Bu modelde base bir convolution layer kullanılarak yüksek accuracy'e ulaşılıp ulaşılamyacağı değerlendirilmek istenmiştir.
Pre-trained model yapıları veya daha fazla hidden layer ekleme seçenekleri değerlendirilebilir.

In [ ]:
# modeli oluşturma
model = Sequential([
    Input(shape=(590, 445, 3)),
    Conv2D(16, (3, 3), activation='relu', kernel_regularizer = l2(0.01)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

In [ ]:
# modeli derleme
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy', 'F1Score', "AUC"])

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
    'best_model.keras',
    save_weights_only=False,
    monitor='val_loss',
    verbose=1,
    save_best_only=True)]

In [ ]:
model.summary()

## Train

In [ ]:
history = model.fit(
    train_ds,
    epochs=7,
    callbacks=callbacks,
    validation_data=val_ds,
    batch_size = 16
)

Bu model yapısı ve epoch sayısı daha önceki denemelerde elde edilen sonuçlara istinaden bu şekilde oluşturulmuştur. Modelin fit edilmesi sonucunda training accuracy: %99 validation accuracy: %92.57 olmuştur. Model checkpoint kullanılması nedeniyle çok yüksek training accuracy elde edilmesine rağmen validation accuracy'nin yüksekliği modelin genelleme yeteneğinin yüksek olduğunu göstermektedir.

## Visualizing Training Process

In [ ]:
# Model sürecinin görselleştirilmesi
def plot_training_history(history, train_loss='loss', train_metric='accuracy', val_loss='val_loss', val_metric='val_accuracy'):

  # Loss görselleştirme
  plt.figure(figsize=(10,5))
  plt.plot(history.history[train_loss], label='Training Loss')
  plt.plot(history.history[val_loss], label='Validation Loss')
  plt.title('Training and Validation Loss Over Epochs')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()

  # Metrikleri görselleştirme
  plt.figure(figsize=(10,5))
  plt.plot(history.history[train_metric], label=f"Training:{train_metric}")
  plt.plot(history.history[val_metric], label=f"Validation:{val_metric}")
  plt.title(f'Training and Validation {train_metric} Over Epochs')
  plt.xlabel('Epochs')
  plt.ylabel(f'train_metric')
  plt.legend()
  plt.show()

In [ ]:
plot_training_history(history)

# Getting Best Epoch Details and Best Parameters

In [ ]:
def get_best_epoch_details(history):
  val_losses = history.history['val_loss']
  min_val_loss_index = val_losses.index(min(val_losses))
  best_epoch = min_val_loss_index + 1

  epoch_details = {}
  for key in history.history.keys():
    epoch_details[key] = history.history[key][min_val_loss_index]

  epoch_details['best_epoch'] = best_epoch
  return epoch_details

In [ ]:
best_epoch_details = get_best_epoch_details(history)
print(f"Best epoch details: {best_epoch_details}")

In [ ]:
saved_model = tf.keras.models.load_model('best_model.keras')
saved_model

# Evaluation

In [ ]:
score = saved_model.evaluate(test_ds,batch_size=256)
print("Accuracy: {:.2f}%".format(score[1] * 100)) 
print("Loss: ",score[0])

Fit etme sonucunda elde edilen yüksek accuracy değerlerden yola çıkarak overfit olma ihtimaline rağmen bağımsız test dataset'i ile modeli evaluate ettiğimizde tutarlı bir sonuç elde edildiği görülmektedir. Model genel anlamda yüksek doğruluk ile balıkları sınıflandırabilmektedir.

## Predictions

In [ ]:
# Tahminler
pred = saved_model.predict(test_ds)
pred_classes = np.argmax(pred, axis=1)

# True class labels
true_classes = []
for _, label in test_ds:
    true_classes.extend(np.argmax(label.numpy(), axis=1))  # Çok sınıflı durumda, argmax ile alıyoruz

true_classes = np.array(true_classes)  # Listeyi numpy dizisine çevir

In [ ]:
# Confusion matrix
conf_matrix = confusion_matrix(true_classes, pred_classes)

class_labels = label_encoder.classes_ 

# Confusion matrix görselleştirme
plt.figure(figsize=(12, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

## Classification Report

In [ ]:
report_dict = classification_report(true_classes, pred_classes, target_names=class_labels, output_dict=True)
report_df = pd.DataFrame(report_dict).transpose()
print(report_df)

## Visualizing Predictions

In [ ]:
images = []
labels = []

# Yeterli örnek almak için test_ds'yi unbatch edin ve listeye ekleyin
for image, label in test_ds.unbatch().take(1000):
    images.append(image.numpy())
    labels.append(np.argmax(label.numpy()))  # one-hot encoded ise argmax ile sınıf indeksini alın

images = np.array(images)
labels = np.array(labels)

# Tahminleri alın
pred = model.predict(test_ds)
pred_classes = np.argmax(pred, axis=1)  # Her tahminin en yüksek olasılıklı sınıfını alın

# Rastgele 15 örnek seçin
indices = random.sample(range(len(images)), 15)
random_images = images[indices]
random_labels = labels[indices]
random_preds = pred_classes[indices]

# Görselleştirme
fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(15, 7), subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(random_images[i])
    if random_labels[i] == random_preds[i]:
        color = 'green'
    else:
        color = 'red'
    ax.set_title(f"True: {class_labels[random_labels[i]]}\nPredicted: {class_labels[random_preds[i]]}", color=color)

plt.tight_layout()
plt.show()

# Conclusion
Rastgele seçilen 15 test örneği üzerinde elde edilen sonuçlara bakıldığında modelin yüksek doğruluk oranıyla tahminleme yapabildiği görülmektedir. 
Farklı yöntemler denenerek (cnn mimarisi yerine ann mimarisi kullanmak gibi) model daha başarılı hale getirilebilir.